In [9]:
import xarray
import pandas as pd
import seaborn
import os
import pickle
import statistics
import numpy as np

from neural_structural_optimization import pipeline_utils
from neural_structural_optimization import problems
from neural_structural_optimization import models
from neural_structural_optimization import topo_api
from neural_structural_optimization import train_switch
import matplotlib.pyplot as plt
from neural_structural_optimization.problems import PROBLEMS_BY_NAME

In [2]:
max_iterations=200
switch = [1,2,4,8,16,32,64]
width = [128,64,32,16,1]
cnn_kwargs=dict(resizes=(1, 1, 2, 2, 1))

In [3]:
examples = PROBLEMS_BY_NAME
example_list =list(examples)
len(set(list(examples)))

116

In [4]:
D={}
D_norm ={}

In [5]:
for name in example_list:
    example = examples[name]
    result = {}
    all=[]

    if os.path.exists("results/cnn_{}.pkl".format(name+"_"+str(max_iterations))):
        ds_cnn_all = pickle.load(open("results/cnn_{}.pkl".format(name+"_"+str(max_iterations)), 'rb'))
        result["CNN"] = ds_cnn_all.loss
    else:
        continue
    if os.path.exists("results/pixel_{}.pkl".format(name)):
        ds_pixel = pickle.load(open("results/pixel_{}.pkl".format(name), 'rb'))
        result["Pixel"] = ds_pixel.loss
    else:
        continue
    all =result['CNN'].to_dict()['data']+result['Pixel'].to_dict()['data']

    for i in switch:
        if os.path.exists("results/pixel_switch_{}.pkl".format(name+"_"+str(i))):
            ds_pixel_switch = pickle.load(open("results/pixel_switch_{}.pkl".format(name+"_"+str(i)), 'rb'))
            result["Pixel_switch_{}".format(i)] = ds_pixel_switch.loss
            all =all + result["Pixel_switch_{}".format(i)].to_dict()['data']
        else:
            continue
    
    D[name]=result
        
#NORMALIZATION
    result_norm={}
    cnn_norm=[]
    max_example = max(all)
    min_example = min(all)
    
    for k in result:
        loss_norm=[]
        for step_loss in result[k]:
            loss_norm.append((float(step_loss)-min_example)/(max_example-min_example))
        result_norm[k]=loss_norm

    D_norm[name]=result_norm

In [6]:
D_norm_min_loss={}
for example in list(D_norm):
    column={}
    for model in list(D_norm[example]):
        column[model]=min(list(D_norm[example][model]))
    D_norm_min_loss[example]=column

In [7]:
table=pd.DataFrame.from_dict(D_norm_min_loss)
table

""


In [32]:
import numpy as np
d = {}
for col in table:
    if any([np.isnan(v) for v in table[col].values]):
        pass
    else:
        d[col] = table[col]
table_filtered = pd.DataFrame(d)

In [33]:
mean_value_column=[]
for model in list(table_filtered.axes[0]):
    mean_val = statistics.mean(list(table_filtered.loc[model,:]))
    mean_value_column.append(mean_val)
    print(model," ",mean_val)

CNN   0.0001076018154699528
Pixel   0.0014210402076380183
Pixel_switch_1   0.0007339869810192303
Pixel_switch_16   0.00014409033925349114
Pixel_switch_2   0.0007281127038674099
Pixel_switch_32   0.00010288571777471654
Pixel_switch_4   0.0006804429597209229
Pixel_switch_64   6.715259873604605e-05
Pixel_switch_8   0.00037299371795827975


In [34]:
init_loss_norm=[]
for model in list(table_filtered.axes[0]):
    init_losses=[]
    for example in list(D_norm):
        if model in list(D_norm[example]):
            init_losses.append(list(D_norm[example][model])[0])
    init_loss_norm.append(statistics.mean(init_losses))
init_loss_norm

['CNN', 'Pixel', 'Pixel_switch_1', 'Pixel_switch_16', 'Pixel_switch_2', 'Pixel_switch_32', 'Pixel_switch_4', 'Pixel_switch_64', 'Pixel_switch_8']
CNN  0.42407221124232714
Pixel  0.11492153447611902
Pixel_switch_1  0.16461749195429462
Pixel_switch_16  0.002437622181137599
Pixel_switch_2  0.11115567186009095
Pixel_switch_32  0.0007176719812474766
Pixel_switch_4  0.05433620416453092
Pixel_switch_64  0.00034141952776915485
Pixel_switch_8  0.01565390873823116


[0.42407221124232714,
 0.11492153447611902,
 0.16461749195429462,
 0.002437622181137599,
 0.11115567186009095,
 0.0007176719812474766,
 0.05433620416453092,
 0.00034141952776915485,
 0.01565390873823116]

In [35]:
table_filtered['Mean Value']=mean_value_column
table_filtered['Mean Initial Loss']=init_loss_norm
table_filtered.sort_values('Mean Value')

,anchored_suspended_bridge_128x128_0.1,anchored_suspended_bridge_64x64_0.15,cantilever_beam_full_192x64_0.3,cantilever_beam_full_96x32_0.4,cantilever_beam_two_point_128x96_0.3,cantilever_beam_two_point_256x192_0.2,cantilever_beam_two_point_64x48_0.4,canyon_bridge_128x128_0.12,canyon_bridge_64x64_0.16,causeway_bridge_low_128x128_0.1,...,roof_128x128_0.15,roof_64x64_0.2,staggered_points_64x64_0.3,staircase_64x64_0.3,thin_support_bridge_128x128_0.2,thin_support_bridge_64x64_0.3,two_level_bridge_128x128_0.16,two_level_bridge_64x64_0.2,Mean Value,Mean Initial Loss
Pixel_switch_64,0.000000e+00,0.000308,1.403327e-10,0.000109,0.000072,3.293665e-10,0.000009,0.000000e+00,0.000131,0.000230,...,0.000002,0.000000,0.000347,0.000046,0.000000,0.000038,0.000009,0.000117,0.000067,0.000341
Pixel_switch_32,4.906556e-07,0.000442,3.072109e-10,0.000238,0.000136,6.362679e-10,0.000083,9.134906e-07,0.000170,0.000239,...,0.000061,0.000186,0.000195,0.000000,0.000026,0.000040,0.000087,0.000166,0.000103,0.000718
CNN,9.709744e-08,0.000266,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,5.381562e-06,0.000062,0.000216,...,0.000000,0.000041,0.000725,0.000254,0.000010,0.000017,0.000000,0.000063,0.000108,0.424072
Pixel_switch_16,9.252855e-07,0.000592,5.195422e-10,0.000406,0.000183,8.344589e-10,0.000616,4.439019e-06,0.000172,0.000305,...,0.000116,0.000303,0.000452,0.000015,0.000182,0.000052,0.000160,0.000249,0.000144,0.002438
Pixel_switch_8,4.538521e-06,0.000858,1.179998e-09,0.000477,0.000277,1.672647e-09,0.000985,1.130448e-05,0.000192,0.000366,...,0.000665,0.000464,0.001630,0.000083,0.000338,0.000083,0.000228,0.000459,0.000373,0.015654
Pixel_switch_4,3.854139e-05,0.000235,2.778529e-09,0.002364,0.000832,2.426215e-09,0.003363,2.924664e-04,0.000223,0.000395,...,0.000602,0.004060,0.003189,0.000830,0.000392,0.000084,0.000384,0.000299,0.000680,0.054336
Pixel_switch_2,3.630911e-05,0.000302,2.475177e-09,0.001409,0.000857,2.625143e-09,0.003464,2.615872e-04,0.000328,0.000503,...,0.000805,0.004706,0.003247,0.000589,0.000643,0.000088,0.000429,0.000521,0.000728,0.111156
Pixel_switch_1,3.630826e-05,0.000301,2.475173e-09,0.001409,0.000857,3.546251e-09,0.003382,2.615865e-04,0.000328,0.000329,...,0.000805,0.004706,0.003246,0.000589,0.000643,0.000002,0.000429,0.000521,0.000734,0.164617
Pixel,8.533476e-06,0.000000,3.065293e-09,0.002332,0.000864,1.959474e-09,0.003852,6.631201e-05,0.000000,0.000000,...,0.002647,0.009665,0.000000,0.001215,0.000897,0.000000,0.000243,0.000000,0.001421,0.114922


In [77]:
table_mean_val={}

for i in table_filtered.axes[0]:
    table_mean_val[i]=round(table_filtered['Mean Value'][i],5)

table_summary = pd.DataFrame.from_dict(table_mean_val,orient='index')


       CNN    Pixel  Pixel_switch_1  Pixel_switch_2  Pixel_switch_4  \
0  0.00011  0.00142         0.00073         0.00073         0.00068   

   Pixel_switch_8  Pixel_switch_16  Pixel_switch_32  Pixel_switch_64  
0         0.00037          0.00014           0.0001          0.00007  
\begin{tabular}{lrrrrrrrrr}
\toprule
{} &      CNN &    Pixel &  Pixel\_switch\_1 &  Pixel\_switch\_2 &  Pixel\_switch\_4 &  Pixel\_switch\_8 &  Pixel\_switch\_16 &  Pixel\_switch\_32 &  Pixel\_switch\_64 \\
\midrule
0 &  0.00011 &  0.00142 &         0.00073 &         0.00073 &         0.00068 &         0.00037 &          0.00014 &           0.0001 &          0.00007 \\
\bottomrule
\end{tabular}



# Summary curve


In [27]:
models = list(D_norm[list(D_norm)[0]])
examples = table_filtered.columns
examples = [e for e in examples if e != 'Mean Value'] 

In [28]:
%matplotlib inline
thresholds = np.linspace(0, 0.01, 50)
for model in models:
    curve = []
    for threshold in thresholds:
        percentage = len([example for example in examples if min(D_norm[example][model]) < threshold]) / float(len(examples))
        curve.append(percentage)
    plt.plot(thresholds, curve, label=model)
plt.xlabel("Relative error threshold")
plt.ylabel("Cumulative probability")
plt.legend()


KeyError: 'anchored_suspended_bridge_128x128_0.1'

# Examples


In [8]:
examples = PROBLEMS_BY_NAME
rng=np.random.RandomState(827)
example_list = list(rng.choice(list(examples),4,replace=False))
example_list

NameError: name 'np' is not defined

In [84]:
for name in example_list:
    
    if os.path.exists("results/cnn_{}.pkl".format(name+"_"+str(max_iterations))):
        ds_cnn = pickle.load(open("results/cnn_{}.pkl".format(name+"_"+str(max_iterations)), 'rb'))
    else:
        continue
    
    if os.path.exists("results/pixel_{}.pkl".format(name)):
        ds_pixel = pickle.load(open("results/pixel_{}.pkl".format(name), 'rb'))
    else:
        continue

    if os.path.exists("results/pixel_switch_{}.pkl".format(name+"_"+str(8))):
        ds_pixel_switch_8 = pickle.load(open("results/pixel_switch_{}.pkl".format(name+"_"+str(8)), 'rb'))
    else:
        continue   
    if os.path.exists("results/pixel_switch_{}.pkl".format(name+"_"+str(64))):
        ds_pixel_switch_64 = pickle.load(open("results/pixel_switch_{}.pkl".format(name+"_"+str(64)), 'rb'))
    else:
        continue  

   
    z_cnn =ds_cnn.design.sel(step=max_iterations)
    z_cnn = z_cnn.to_masked_array().data.reshape(1, z_cnn.shape[0], z_cnn.shape[1])
    plt.imshow(z_cnn[0])
    plt.savefig("eeml_results/"+"cnn_"+name+".pdf")
    plt.close()
    loss_cnn_all = float(ds_cnn_all.loss[max_iterations])
    
    step_pixel =(len(ds_pixel.design.loc[:,0].step)-1)
    z_pixel = ds_pixel.design.sel(step=step_pixel)
    z_pixel = z_pixel.to_masked_array().data.reshape(1, z_pixel.shape[0], z_pixel.shape[1])
    plt.imshow(z_pixel[0])
    plt.savefig("eeml_results/"+"pixel_"+name+".pdf")
    plt.close()
    
    step_pixel_switch_8 =(len(ds_pixel_switch_8.design.loc[:,0].step)-1)
    z_pixel_switch_8 = ds_pixel_switch_8.design.sel(step=step_pixel_switch_8)
    z_pixel_switch_8 = z_pixel_switch_8.to_masked_array().data.reshape(1, z_pixel_switch_8.shape[0], z_pixel_switch_8.shape[1])
    plt.imshow(z_pixel_switch_8[0])
    plt.savefig("eeml_results/"+"pixel_switch_8_"+name+".pdf")
    plt.close()    

    step_pixel_switch_64 =(len(ds_pixel_switch_64.design.loc[:,0].step)-1)
    z_pixel_switch_64 = ds_pixel_switch_64.design.sel(step=step_pixel_switch_64)
    z_pixel_switch_64 = z_pixel_switch_64.to_masked_array().data.reshape(1, z_pixel_switch_64.shape[0], z_pixel_switch_64.shape[1])
    plt.imshow(z_pixel_switch_64[0])
    plt.savefig("eeml_results/"+"pixel_switch_64_"+name+".pdf")
    plt.close()        
    
    
    from matplotlib.pyplot import figure
    figure(num=None, figsize=(8, 3.5), dpi=80, facecolor='w', edgecolor='k')
    
    print(ds_cnn.loss.to_pandas().cummin().plot(linewidth=2, label="CNN"))
    print(ds_pixel.loss.to_pandas().cummin().plot(linewidth=2, label="Pixel"))
    print(ds_pixel_switch_8.loss.to_pandas().cummin().plot(linewidth=2, label="Switch-8"))
    print(ds_pixel_switch_64.loss.to_pandas().cummin().plot(linewidth=2, label="Switch-64"))
     

    plt.legend()
    plt.yscale("log")
    plt.ylabel('Compliance (loss)')
    plt.xlabel('Optimization step')
    plt.savefig("eeml_results/"+name+"_losses"+".pdf")
    plt.close()
    seaborn.despine()


AxesSubplot(0.125,0.125;0.775x0.755)
AxesSubplot(0.125,0.125;0.775x0.755)
AxesSubplot(0.125,0.125;0.775x0.755)
AxesSubplot(0.125,0.125;0.775x0.755)
